# Multi-Agent Automated Sales Email System
### Using Google Agent Development Kit (ADK)

This notebook demonstrates a multi-agent workflow using Google's ADK.

Architecture:

Sales Agents → Sales Manager → Email Manager → Tools (Subject + HTML + Send Email)

In [ ]:
import os
import asyncio
from typing import Dict, List

# Gemini’s Agent Development Kit
from google import genai
# types is a module containing structured data classes used by the Gemini SDK
from google.genai import types

from dotenv import load_dotenv
import brevo_python
from brevo_python.api.transactional_emails_api import TransactionalEmailsApi
from brevo_python.models.send_smtp_email import SendSmtpEmail

### Setup the API client to Gemini

In [ ]:
load_dotenv(override=True)
# Initialize the new Client
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

# Client(
#     api_key="your_key",
#     http_options=...,   # optional networking config
# )

# returns:-
# client.models.generate_content()     # synchronous
# client.aio.models.generate_content()    #async calls

In [ ]:
# Make a simple test request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Tell a joke in one sentence."
)

# generate_content sends a request to Gemini model
# takes in the model, contents (system instructions) and config (tools, temperature etc.)

# Print the response
print(response.text)

What do you call a fake noodle? An impasta!


### Setup the email sender

In [30]:
# --- Tool Definition ---
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Sends a transactional email via Brevo.
    
    Args:
        subject: The subject line of the email.
        html_body: The HTML content of the email.
    """
    configuration = brevo_python.Configuration()
    configuration.api_key["api-key"] = os.getenv("BREVO_API_KEY")
    api_client = brevo_python.ApiClient(configuration)
    email_api = TransactionalEmailsApi(api_client)
    
    email = SendSmtpEmail(
        sender={"name": "Ayushi Saxena", "email": "khonthedark@gmail.com"},
        to=[{"email": "windwatcher17@gmail.com", "name": "Receiver"}],
        subject=subject,
        html_content=html_body
    )
    try:
        email_api.send_transac_email(email)
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

# List of available functional tools
tools_list = [send_email]

### Agent Workflow

In [31]:
# System Instructions
instructions = {
    "professional": "You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.",
    "engaging": "You are a humorous, engaging sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write witty, engaging cold emails that are likely to get a response.",
    "busy": "You are a busy sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write concise, to the point cold emails.",
    "sales_picker": "You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to. Do not give an explanation; reply with the selected email only.",
    "manager": """You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the email_manager_handoff tools.
        Follow these steps carefully:
        1. Generate Drafts: Use all three email draft tools (get_professional_draft, get_engaging_draft, get_busy_draft) to generate three different email drafts. Do not proceed until all three drafts are ready.
        2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
        3. Use the email_manager_handoff tool to send the best email (and only the best email) to the user.
        Crucial Rules:
        - You must use the get draft tools to generate the drafts — do not write them yourself.
        - You must send ONE email using the email_manager_handoff tool — never more than one.""",
    "subject": "You write email subjects likely to get a response based on the message provided.",
    "html": "You convert text email bodies to clear, compelling HTML layouts.",
}

### Agent Wrappers (Tools)

Gemini uses function calling to simulate "agents calling agents"

In [ ]:
async def get_professional_draft(request: str) -> str:
    """Generate a professional cold sales email draft."""
    # Await an asynchronous API call to Gemini
    # `client.aio` is the async version of the SDK client
    res = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(system_instruction=instructions["professional"]),
        contents=request
    )
    return res.text

In [33]:
async def get_engaging_draft(request: str) -> str:
    """Generate a witty and engaging cold sales email draft."""
    res = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(system_instruction=instructions["engaging"]),
        contents=request
    )
    return res.text

In [34]:
async def get_busy_draft(request: str) -> str:
    """Generate a concise and busy cold sales email draft."""
    res = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(system_instruction=instructions["busy"]),
        contents=request
    )
    return res.text

In [35]:
async def email_manager_handoff(email_body: str) -> str:
    """
    Handoff the winning email draft to the Email Manager for formatting and sending.
    Args:
        email_body: The text of the winning email.
    """
    print(f"\n--- Handoff to Email Manager ---\n")
    
    # 1. Get Subject
    subj_res = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(system_instruction=instructions["subject"]),
        contents=f"Write a subject for: {email_body}"
    )
    subject = subj_res.text

    # 2. Convert to HTML
    html_res = await client.aio.models.generate_content(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(system_instruction=instructions["html"]),
        contents=email_body
    )
    html_content = html_res.text

    # 3. Send via Brevo
    send_email(subject=subject, html_body=html_content)
    return "Email formatted and sent successfully."

### Simulate Running Agents Concurrently

In [36]:
message = "Write a cold sales email."

draft_tasks = [
    get_professional_draft(message),
    get_engaging_draft(message),
    get_busy_draft(message)
]

drafts = await asyncio.gather(*draft_tasks)

print(drafts)

["Subject: Streamline Your SOC 2 Compliance & Audit Prep with AI | ComplAI\n\nDear [Prospect's Name],\n\nI'm reaching out from ComplAI because I understand that navigating the complexities of SOC 2 compliance and preparing for audits can be a significant challenge for growing organizations like [Prospect's Company Name]. The manual effort, potential for oversight, and the constant demand on internal resources often divert focus from core business objectives.\n\nThis is precisely where ComplAI offers a transformative solution. We provide an AI-powered SaaS platform specifically designed to simplify and accelerate your SOC 2 journey. Our tool automates critical compliance processes, proactively identifies gaps, streamlines evidence collection, and ensures you are continuously audit-ready – all with unparalleled efficiency and accuracy.\n\nImagine reducing the time, cost, and stress traditionally associated with SOC 2, while simultaneously enhancing your security posture and demonstrating

In [39]:
async def run_manager_agent(message: str):

    manager_tools = [get_professional_draft, get_engaging_draft, get_busy_draft, email_manager_handoff]

    # Map tool name → function
    tool_map = {
        "get_professional_draft": get_professional_draft,
        "get_engaging_draft": get_engaging_draft,
        "get_busy_draft": get_busy_draft,
        "email_manager_handoff": email_manager_handoff
    }

    # contents = message
    conversation = [message]

    while True:

        response = await client.aio.models.generate_content(
            model="gemini-2.5-flash",
            config=types.GenerateContentConfig(
                system_instruction=instructions["manager"],
                tools=manager_tools,
                automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True) 
            ),
            contents=conversation
        )

        candidate = response.candidates[0]
        parts = candidate.content.parts
        conversation.append(candidate.content)

        tool_called = False

        for part in parts:
            # MODEL IS REQUESTING TOOL EXECUTION
            if hasattr(part, "function_call") and part.function_call:
                tool_called = True
                tool_name = part.function_call.name
                tool_args = dict(part.function_call.args)
                print(f"Tool requested: {tool_name}")
                print(f"Arguments: {tool_args}")
                # Execute tool
                tool_fn = tool_map[tool_name]
                result = await tool_fn(**tool_args)
                print(f"Tool result: {result}")
                # Append tool response
                conversation.append(
                    types.Content(
                        role="tool",
                        parts=[
                            types.Part.from_function_response(
                                name=tool_name,
                                response={"result": result}
                            )
                        ]
                    )
                )
                break

        # If no tool requested → final answer
        if not tool_called:
            print("\nFINAL MANAGER OUTPUT:\n")
            print(response.text)
            return response.text

In [41]:
async def main():
    message = "Send out a cold sales email addressed to Dear CEO from Ayushi Saxena"
    print("Sales Manager evaluating and handing off...")
    await run_manager_agent(message)

await main()


Sales Manager evaluating and handing off...


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 9.942071372s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 'global'}, 

<hr>